<h1> Ćwiczenia 6 - Programowanie w GIS (QGIS)- warstwy, projekty, podstawowe funkcjonalności </h1>

Na wykładzie poruszyliśmy już 5 podstawowych klas, które będziemy używać:

<ul>
    <li>QgsProject - klasa projektu </li>
    <li>QgsVectorLayer - klasa warstwy wektorowej </li>
    <li>QgsRasterLayer - klasa warstwy rastrowej </li>
    <li>QgsMapLayerRegistry - rejestr warstw </li>
    <li>iface - klasa interfejsu aplikacji </li>
</ul>

Rozpocznijmy od przetestowania kilku podstawowych funkcjonalności API.

Zawsze zaczynamy od importu biblioteki core, zawierającej podstawowe funkcjonalności.



In [ ]:
from qgis.core import *

Następnie możemy stworzyć instancję obecnie otwartego projektu.

In [ ]:
projekt = QgsProject.instance()
projekt.fileName() # nazwa projektu, obecnie pusta
sciezka = """C://sciezka//do//pliku"""
projekt.write(sciezka) #zapis projektu
projekt.read(sciezka) #wczytanie projektu

<h3>Warstwy wektorowe</h3>
Dodamy warstwę wektorową.
Zwróć uwagę na wykorzystanie biblioteki os.

In [ ]:
import os
vlayer = QgsVectorLayer(os.path.join(wektor_folder,"Drogi_JG.shp"),"drogi","ogr") #ładowanie warstwy do zmienne
#to nie dodaje warstwy do mapy!
rejestr = QgsMapLayerRegistry.instance()
rejestr.addMapLayer(vlayer) #to dodaje!

Lub wprost do rejestru:

In [ ]:
iface.addVectorLayer(os.path.join(wektor_folder,"Budynki_JG.shp"),"drogi","ogr")

Usuwanie:

In [ ]:
rejestr.removeMapLayer(vlayer) #jesli jest w zmiennej


rejestr.removeMapLayer("id") #musimy odnalezc id jesli dodalismy ja przez iface.
rejestr.mapLayers() #za pomocą tego polecenia

#Jak dodac wszystkie warstwy z folderu?

<h3>Warstwy rastrowe</h3>
Wszystko odbywa się analogicznie, z rożnicą, że nie ma potrzeby definicji sterownika obsługującego warstwę.

In [ ]:
iface.addRasterLayer("X:\\GitHub\\PwSGIS\\Ćwiczenia\\QGISRaster\\bogota.tif","Bogota")

In [ ]:
rlayer = iface.addRasterLayer("X:\\GitHub\\PwSGIS\\Ćwiczenia\\QGISRaster\\bogota.tif","Bogota") #iface zwraca wskaznik do zmiennej, wiec mozna to wykorzystac by potem ja wygodnie usunac
rejestr.removeMapLayer(rlayer)

<h4>Szczegóły warstwy - raster </h4>

Dostęp do danych, wartości, operacje

<b>Podstawowe właściwości:</b>

In [ ]:
rlayer.width()

rlayer.height()

rlayer.extent()

rlayer.extent().toString()
rlayer.rasterType() #0 = Skala szarosci, 1 = (single band), 2 = Multiband

rlayer.isValid()

rlayer.bandCount()

rlayer.metadata()


Odczyt wartości piksela:

In [ ]:
ident = rlayer.dataProvider().identify(QgsPointXY(453138,81225), QgsRaster.IdentifyFormatValue)
ident.results()

Sporo się tu wydarzyło. Przedewszystkim rlayer to nasza warstwa. dataProvider() to nowa klasa - klasa przechowująca dane w warstwie - wartości pikseli lub wartości atrybutów w warstwach wektorowych. Posiada ona metodę identify (patrz dokumentacja). Jej syntax to:

identify(QgsPoint, format)

Qgis korzysta z własnych typów danych - tutaj jest to punkt QGIS, dlatego współrzędne należało definiować w tym typie danych.

<h4> Szczegóły warstwy - wektor </h4>

Dostęp do danych, wartości operacje:

In [ ]:
#Ładujemy dane
drogi = iface.addVectorLayer(os.path.join(wektor_folder,"Drogi_JG.shp"),"","ogr")
budynki = iface.addVectorLayer(os.path.join(wektor_folder,"Budynki_JG.shp"),"drogi","ogr")
#Przegląd atrybutów
for pole in drogi.fields():
	print(pole.name(), pole.typeName())


In [ ]:
#Możemy wybrać aktywną warstwę i przypisać ją pod zmienną:
layer = iface.activeLayer()

In [ ]:
#Pozyskanie obiektów na warstwie
layer.getFeatures() #zwraca iterator, nie listę, można się dopiero przez nią iterować
features = [feature for feature in drogi.getFeatures()][:20]

In [ ]:
for obiekt in features:
    print(obiekt.id())
    
for obiekt in features:
    print(obiekt.geometry()) #Zwraca klasę geometrii

for obiekt in features:
    print(obiekt.geometry().length())

for obiekt in features:
    print(obiekt.attributes())

for obiekt in features:
    print(obiekt['highway'])


Selekcja:

In [ ]:
drogi.selectAll()

drogi.selectByExpression(""" "highway" = 'secondary' """)

In [ ]:
iface.mapCanvas() #klasa płótna, czyli wyświetlanej mapy, pozwala na edycję wyświetlania

drogi.selectAll()
iface.mapCanvas().setSelectionColor( QColor("magenta"))

#Lub w HEX RGB
iface.mapCanvas().setSelectionColor( QColor("#FFFF00"))

Iteracja po selekcji

In [ ]:
drogi.selectByExpression(""" "highway" = 'secondary'""")
selection = drogi.selectedFeatures()
print(len(selection))
for feature in selection:
    print(feature.id())


<h3> To wszystko na dziś! Wykonaj zadania z Listy 6 </h3>